In [131]:
import json
import glob
import os
import datetime
import re
import pandas as pd
from slack_export import SlackExport, normalize_links


In [4]:
data_folder = './ODS_dump_Mar_10_2017'
ods = SlackExport(data_folder)

In [7]:
df_msg = pd.DataFrame.from_records(ods.messages)
df_msg.fillna(0).groupby(['type', 'subtype'])['dt'].count()

type     subtype          
message  0                    301495
         bot_add                  11
         bot_message            2582
         bot_remove                6
         channel_archive          27
         channel_join          61056
         channel_leave          2564
         channel_name             39
         channel_purpose          83
         channel_topic           141
         channel_unarchive         1
         file_comment            396
         file_mention             53
         file_share             3189
         me_message                8
         pinned_item             380
         reminder_add              8
         reply_broadcast         184
         sh_room_created           1
         tombstone                33
Name: dt, dtype: int64

In [18]:
df_msg.subtype.unique()

array([u'channel_join', nan, u'channel_leave', u'file_share',
       u'channel_topic', u'file_comment', u'pinned_item',
       u'reply_broadcast', u'file_mention', u'channel_purpose',
       u'channel_name', u'bot_add', u'bot_remove', u'me_message',
       u'channel_archive', u'bot_message', u'reminder_add',
       u'sh_room_created', u'tombstone', u'channel_unarchive'], dtype=object)

In [20]:
# %load slack_export.py
import json
import glob
import os
import datetime
import re


def _read_json_dict(filename, key='id'):
    with open(filename) as fin:
        records = json.load(fin)
        json_dict = {
            record[key]: record
            for record in records
        }
    return json_dict


class SlackExport(object):
    def __init__(self, export_path):
        self.load_export(export_path)

    def load_export(self, export_path):
        self.channels = _read_json_dict(os.path.join(export_path, 'channels.json'))
        self.users = _read_json_dict(os.path.join(export_path, 'users.json'))
        self.messages = []
        for channel_id, channel in self.channels.iteritems():
            messages_glob = os.path.join(export_path, channel['name'], '*.json')
            for messages_filename in glob.glob(messages_glob):
                with open(messages_filename) as f_messages:
                    for record in json.load(f_messages):
                        if 'ts' in record:
                            record['ts'] = float(record['ts'])
                            record['dt'] = datetime.datetime.fromtimestamp(record['ts'])
                        record['channel'] = channel_id
                        self.messages.append(record)

re_slack_link = re.compile(r'(?P<all><(?P<id>[^\|]*)(\|(?P<title>[^>]*))?>)')

def _extract_slack_link_id(m):
    return m.group('id')

def normalize_links(text):
    return re_slack_link.sub(_extract_slack_link_id, text)

In [21]:
class WelcomeExport(object):
    def __init__(self, export_path):
        self.load_export(export_path)

    def load_export(self, export_path):
        self.users = _read_json_dict(os.path.join(export_path, 'users.json'))
        self.messages = []
        messages_glob = os.path.join(export_path, 'welcome', '*.json')
        for messages_filename in glob.glob(messages_glob):
            with open(messages_filename) as f_messages:
                for record in json.load(f_messages):
                    if 'ts' in record:
                        record['ts'] = float(record['ts'])
                        record['dt'] = datetime.datetime.fromtimestamp(record['ts'])
                    record['channel'] = 'welcome'
                    self.messages.append(record)


In [22]:
ods = WelcomeExport(data_folder)

In [25]:
df_msg = pd.DataFrame.from_records(ods.messages)
df_msg.fillna(0).groupby(['type', 'subtype'])['dt'].count()

type     subtype        
message  0                  3214
         channel_join       2987
         channel_leave        95
         channel_purpose       1
         channel_topic         1
         file_share            4
         reply_broadcast       1
         tombstone             2
Name: dt, dtype: int64

In [37]:
print df_msg[df_msg.subtype.isnull()].text[4]

Всем привет. Меня зовут Артём, я закончил Иркутский политех по специальности радиотехника. Сейчас я аспирант, занимаюсь обработкой сигналов атмосферных радаров. Недавно заинтересовался машинным обучением, прошел курс Andrew Ng, сейчас прохожу курс от Яндекса и МФТИ на курсере. Поражает количество практических задач, которые позволяет решать data science, поэтому я здесь.


Вытащим сообщения-представления

In [128]:
k = 0
hi_messages = []
hi_tokens=(u'все', u'привет',u'добр',u'шалом',u'салют',u'здрав',u'хай',u'я',u'ребят'u'коллег')
for _, i in df_msg[df_msg.subtype.isnull()].iterrows():
    if i.text.lower().startswith(hi_tokens) and len(i.text) > 100:
        k+=1
        hi_messages.append((i.username, i.text))

In [129]:
for i in hi_messages:
    print i[1]
    print '='*80

Всем привет. Меня зовут Артём, я закончил Иркутский политех по специальности радиотехника. Сейчас я аспирант, занимаюсь обработкой сигналов атмосферных радаров. Недавно заинтересовался машинным обучением, прошел курс Andrew Ng, сейчас прохожу курс от Яндекса и МФТИ на курсере. Поражает количество практических задач, которые позволяет решать data science, поэтому я здесь.
Всем привет!
Меня зовут Егор, я из Питера, учусь в Физико-Технической школе при Академическом Университете.
Недавно отвлекся от спортивного программирования и заинтересовался машинным обучением. После того, как посмотрел курс Deep Learning from Google на Udacity и поразбирался с семплами, решил попробовать классифицировать музыку сверточными нейросетями. Пишу на питоне с keras, учусь всякому-разному :slightly_smiling_face:
я сделал нейросетку, которая определяет жанр мелодии по ее 60-секундному отрывку. Пока что только со сверточными и полносвязными слоями, сейчас прикручиваю lstm. Не самая оригинальная задача :slightl

In [130]:
k, len(df_msg[df_msg.subtype.isnull()].text)

(612, 3214)